# LAB Kmeans _ Latin Playlists

1. take the vastness of songs you have collected together with their audio features and train a k-means model with them - similar how we did in lecture (scaling is important!). Keep in mind, that it doesn't make sense to train a model, when you have artist or title name or track id in the data. You need to however have the possibility to later (after your model has determined the cluster ids) reassign the correct track ids to the rows. You can do this by just adding back the track id to the dataframe that contains the determined cluster id's, but of course only if you haven't shuffled your data rowwise. You can optimize a bit the parameters, but for a first iteration of your model, just go with the default values of KMeans and stick with it. Only improve later by tuning the parameters.

In [77]:
# get data from our song database

import pandas as pd

data = pd.read_csv('/Users/josephinebiedermann/Desktop/DABC2021/GitHub/ProjectsPHine/Week7/Week7Project/data/audiofeatures_latin_playlists_merged.csv')
data.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,...,80.015,audio_features,2rCbl9naJYhaxjLsfx88uM,spotify:track:2rCbl9naJYhaxjLsfx88uM,https://api.spotify.com/v1/tracks/2rCbl9naJYha...,https://api.spotify.com/v1/audio-analysis/2rCb...,195053,4,Pareja Del Año,Sebastian Yatra
1,1,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,...,80.015,audio_features,2rCbl9naJYhaxjLsfx88uM,spotify:track:2rCbl9naJYhaxjLsfx88uM,https://api.spotify.com/v1/tracks/2rCbl9naJYha...,https://api.spotify.com/v1/audio-analysis/2rCb...,195053,4,Pareja Del Año,Sebastian Yatra
2,2,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,...,80.015,audio_features,2rCbl9naJYhaxjLsfx88uM,spotify:track:2rCbl9naJYhaxjLsfx88uM,https://api.spotify.com/v1/tracks/2rCbl9naJYha...,https://api.spotify.com/v1/audio-analysis/2rCb...,195053,4,Pareja Del Año,Sebastian Yatra
3,3,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,...,80.015,audio_features,2rCbl9naJYhaxjLsfx88uM,spotify:track:2rCbl9naJYhaxjLsfx88uM,https://api.spotify.com/v1/tracks/2rCbl9naJYha...,https://api.spotify.com/v1/audio-analysis/2rCb...,195053,4,Pareja Del Año,Sebastian Yatra
4,4,0.807,0.712,1,-5.757,1,0.100,0.532,0.000024,0.1630,...,100.004,audio_features,5RAIMjdrCEjpjaR5tBATXU,spotify:track:5RAIMjdrCEjpjaR5tBATXU,https://api.spotify.com/v1/tracks/5RAIMjdrCEjp...,https://api.spotify.com/v1/audio-analysis/5RAI...,241659,4,Aloha,Maluma


In [78]:
# checking the column names to se what we are working with

data.columns

Index(['Unnamed: 0', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'title', 'artist'],
      dtype='object')

In [79]:
# split out the artist name and track id and name as suggested

data_clean = data.drop(['Unnamed: 0', 'type', 'id','uri','track_href','analysis_url', 'title', 'artist'], axis=1)

In [80]:
# we have 1301 songs and 13 features

data_clean.shape

(1620, 13)

In [81]:
# checking the df again

data_clean.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,0.808,80.015,195053,4
1,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,0.808,80.015,195053,4
2,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,0.808,80.015,195053,4
3,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,0.808,80.015,195053,4
4,0.807,0.712,1,-5.757,1,0.100,0.532,0.000024,0.1630,0.792,100.004,241659,4


In [82]:
# data clean is our X, y we dont have

X = data_clean
X.head()

# we can see here that the scales are also very different for the features (e.g. tempo VS energy)
# need to use the scaler

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,0.808,80.015,195053,4
1,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,0.808,80.015,195053,4
2,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,0.808,80.015,195053,4
3,0.715,0.622,4,-5.102,0,0.046,0.163,0.000000,0.0948,0.808,80.015,195053,4
4,0.807,0.712,1,-5.757,1,0.100,0.532,0.000024,0.1630,0.792,100.004,241659,4


In [83]:
# importing StandardScaler

from sklearn.preprocessing import StandardScaler

In [84]:
# applying StandardScaler to X and assign to X prep variable

X_prep = StandardScaler().fit_transform(X) # fit the StandardScaler on X and transform X

In [85]:
X_prep

# this is an numpy array, we need a df

array([[ 0.12069092, -0.20718225, -0.44609764, ..., -1.21975041,
        -0.35047808,  0.16291381],
       [ 0.12069092, -0.20718225, -0.44609764, ..., -1.21975041,
        -0.35047808,  0.16291381],
       [ 0.12069092, -0.20718225, -0.44609764, ..., -1.21975041,
        -0.35047808,  0.16291381],
       ...,
       [-0.57399341,  0.45256131, -1.01357168, ..., -0.62098103,
        -0.58761373,  0.16291381],
       [-0.68086793, -0.38012474, -1.58104572, ..., -0.50228502,
        -0.73339961,  0.16291381],
       [ 0.05198587,  1.19557348, -1.01357168, ..., -0.02385523,
         0.54369435,  0.16291381]])

In [86]:
column_names = list(data_clean.columns)
column_names

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature']

In [87]:
# turn it into df and assigning it to variable X_prep_df

X_prep_df = pd.DataFrame(X_prep, columns = column_names)
X_prep_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.120691,-0.207182,-0.446098,0.404455,-1.125049,-0.612197,-0.545641,-0.197880,-0.581344,0.899200,-1.219750,-0.350478,0.162914
1,0.120691,-0.207182,-0.446098,0.404455,-1.125049,-0.612197,-0.545641,-0.197880,-0.581344,0.899200,-1.219750,-0.350478,0.162914
2,0.120691,-0.207182,-0.446098,0.404455,-1.125049,-0.612197,-0.545641,-0.197880,-0.581344,0.899200,-1.219750,-0.350478,0.162914
3,0.120691,-0.207182,-0.446098,0.404455,-1.125049,-0.612197,-0.545641,-0.197880,-0.581344,0.899200,-1.219750,-0.350478,0.162914
4,0.823009,0.369293,-1.297309,0.140231,0.888851,0.033508,1.025590,-0.197667,-0.027224,0.827820,-0.622415,0.674178,0.162914
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,-0.322075,-0.405746,1.256324,1.048675,0.888851,-0.807104,2.315788,-0.197880,0.663395,0.359390,0.334952,-0.475927,0.162914
1616,-0.016719,-0.617120,0.688850,0.000657,0.888851,-0.874066,-0.038930,-0.197880,-0.539094,1.001808,-0.444252,-0.685823,0.162914
1617,-0.573993,0.452561,-1.013572,-0.732713,0.888851,-0.785581,-0.940365,0.113701,0.346523,-0.033199,-0.620981,-0.587614,0.162914
1618,-0.680868,-0.380125,-1.581046,-0.372079,0.888851,-0.877653,-0.961230,-0.197880,-0.417220,0.470921,-0.502285,-0.733400,0.162914


### start modeling

In [88]:
# importing KMeans

from sklearn.cluster import KMeans

In [89]:
# building an empty estimator, we say 4 clusters, placing 4 centroids somewhere
# randomnes fixed by random state, if we dont have this, we will get different results with each try

kmeans = KMeans(n_clusters=4, random_state= 1234) 

In [90]:
# training the model with our X_prep_df

kmeans.fit(X_prep_df)

# model is now trained and smart

KMeans(n_clusters=4, random_state=1234)

In [91]:
# have a look at the cluster centers

kmeans.cluster_centers_

array([[-1.07589714, -1.31122331, -0.30330791, -1.14360138,  0.22844855,
        -0.37559745,  1.10556139, -0.08579728, -0.02700721, -1.0172309 ,
        -0.0682925 ,  0.08710925, -0.57712264],
       [ 0.36805892,  0.09514892,  0.27156292,  0.15585805, -1.02664958,
         0.28570062, -0.2465678 , -0.16547793, -0.15779535,  0.09928956,
        -0.19538534, -0.1160203 ,  0.15640111],
       [ 0.15175694,  0.50649336, -0.11264236,  0.43880639,  0.85842908,
        -0.06080784, -0.26627845, -0.13987792,  0.16951756,  0.39845609,
         0.21518989,  0.02402598,  0.12667097],
       [ 0.13680692,  0.28157599,  0.03467897, -0.94326242, -0.2299823 ,
        -0.54114933, -0.35677627,  6.12856092, -0.19489044, -0.31762778,
        -0.04041716,  0.7917116 , -0.05924138]])

In [92]:
# calculating interta for this model

kmeans.inertia_

# quite high it seems, but need something to compare against

15441.398647348587

In [93]:
# model is now set up with 4 cluster
# can take now our features and see what cluster our model assignes the songs to

clusters = kmeans.predict(X_prep)
clusters

array([1, 1, 1, ..., 2, 2, 2], dtype=int32)

In [94]:
len(clusters)

# this are the 1301 songs from the imported csv

1620

In [95]:
# investiate the clusters

pd.Series(clusters).value_counts().sort_index()

# distribution: right skewd

0    308
1    613
2    663
3     36
dtype: int64

In [96]:
# put in the cluster per song as a new column

data_clean['cluster'] = clusters
data_clean

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,1
1,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,1
2,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,1
3,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,1
4,0.807,0.712,1,-5.757,1,0.1000,0.5320,0.000024,0.1630,0.792,100.004,241659,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,0.657,0.591,10,-3.505,1,0.0297,0.8350,0.000000,0.2480,0.687,132.041,189347,4,2
1616,0.697,0.558,8,-6.103,1,0.0241,0.2820,0.000000,0.1000,0.831,105.966,179800,4,2
1617,0.624,0.725,2,-7.921,1,0.0315,0.0703,0.034600,0.2090,0.599,100.052,184267,4,2
1618,0.610,0.595,0,-7.027,1,0.0238,0.0654,0.000000,0.1150,0.712,104.024,177636,4,2


## improving our model

### inertia

In [22]:
# defining the function to get the best cluster number size looking at inertia

def get_intertia(n_clusters):
    # build model
    kmeans = KMeans(n_clusters=n_clusters,random_state=1234)
    
    # train model
    kmeans.fit(X_prep)
    
    # calculate intertia
    print(n_clusters,'clusters - intertia:', kmeans.inertia_)

cluster_range = range(1,21) # 1-20 clusters

dct = {cluster_number:get_intertia(cluster_number) for cluster_number in cluster_range}
dct

In [23]:
def get_intertia2(n_clusters):
    # build model
    kmeans = KMeans(n_clusters=n_clusters,random_state=1234)
    
    # train model
    kmeans.fit(X_prep)
    
    # calculate intertia & return it
    return kmeans.inertia_

In [26]:
cluster_range = range(1,41) # 1-10 clusters

lst = [get_intertia2(cluster_number) for cluster_number in cluster_range]
lst

[16913.000000000007,
 14563.177631952653,
 13507.34016188575,
 12407.012436971598,
 11522.106134497817,
 10822.935036764706,
 10273.69461867578,
 9890.882832970465,
 9496.827616511442,
 9203.576956547895,
 8974.380455973433,
 8780.600811293209,
 8568.145388089353,
 8438.212096112846,
 8328.759531854408,
 8123.236900345751,
 8026.2172825704765,
 7936.266018368601,
 7797.086914270239,
 7688.473199623565,
 7592.3024958694,
 7507.941936474933,
 7372.624510390741,
 7275.96820226404,
 7167.966671774904,
 7082.565934843622,
 7017.097627783868,
 6969.567540597737,
 6865.260798800553,
 6832.512618463517,
 6769.210930451806,
 6700.618840298284,
 6598.249482669912,
 6532.873174783498,
 6461.552394448923,
 6405.76694141637,
 6368.864015095286,
 6303.900703787245,
 6260.704916003384,
 6211.558122580209]

# make a plot for the intertia scores of the cluster numbers

import matplotlib.pyplot as plt

plt.figure(figsize=(16,8))
plt.plot(cluster_range, lst , 'bo-')

### silhouette

In [28]:
from sklearn.metrics import silhouette_score

In [29]:
# on model that we have trained on the audio

# with cluster numbers ranging from 1 to 40

K = range(2,41)

silhouettes = [] # collect silhouette scores per clusternumber in list

for k in K:
    # build model
    kmeans = KMeans(n_clusters = k, random_state = 1234)
    
    # train model
    kmeans.fit(X_prep)
    
    # calculate silhouette score and append to list
    silhouettes.append(silhouette_score(X_prep, kmeans.predict(X_prep)))

In [30]:
silhouettes

[0.1786070159889639,
 0.11296787479292048,
 0.12258492502033953,
 0.1334739247117646,
 0.13473139338287438,
 0.13137170827448144,
 0.12668896861952678,
 0.12718624487929403,
 0.11772617322549361,
 0.11502790832342251,
 0.11361546985248112,
 0.1191817417471068,
 0.1140972760770522,
 0.10928210077271151,
 0.1121895329298935,
 0.10710738612657107,
 0.10657619926073592,
 0.11298217444894453,
 0.10624324767969623,
 0.10590434631314923,
 0.10472925753850658,
 0.10932752608547938,
 0.11622982937404673,
 0.10591891607359827,
 0.10923755764637455,
 0.10500601674822242,
 0.10749779520129792,
 0.1179123619028909,
 0.11514933428834907,
 0.1124171969604261,
 0.11074944181900098,
 0.10670889499257688,
 0.10865169832588809,
 0.10950232334993337,
 0.11222790907284722,
 0.111268303143674,
 0.1148572379154182,
 0.11319665143024618,
 0.11424507758409287]

import matplotlib.pyplot as plt

plt.figure(figsize=(16,8))
plt.plot(K, silhouettes, 'bo-')
plt.xlabel('k (number od clusters)')
plt.ylabel('silhouette value')

In [33]:
 #pip install yellowbrick

     |████████████████████████████████| 271 kB 24 kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


from yellowbrick.cluster import SilhouetteVisualizer

fig, ax = plt.subplots(4, 2, figsize=(15,8))
for k in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    '''
    Create KMeans instance for different number of clusters
    '''
    km = KMeans(n_clusters=k,
                random_state=1234)
    q, mod = divmod(k, 2)
    '''
    Create SilhouetteVisualizer instance with KMeans instance
    Fit the visualizer
    '''
    
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(X_prep)

In [114]:
# looks like the 2 clusters habe the best silhoutte value
# also all clusters are over the avg silhoutte score and only one cluster goes in the negativ

3. "Predicting" which cluster id a song belongs to, would work like this: take the audio-features of the song and put it into a numpy array, let's say song_array. The audio feature order must exactly match the order that the model has learned. Call the model with KMeans.predict(song_array) to make the prediction. The outcome is only one cluster
4. The recommendation would then work like this: Take the determined cluster id from 2. and subset your spotipy dataframe to filter for this cluster with e.g. df[df["cluster_id'']==2] and get a random song back from that subset. You can take a look at pandas.DataFrame.sample



### Decided to go with 2 clusters, based on the visualisations and the average silouette score. 

In [97]:
kmeans = KMeans(n_clusters=2, random_state= 1234) 

In [98]:
kmeans.fit(X_prep_df)

KMeans(n_clusters=2, random_state=1234)

In [99]:
kmeans.cluster_centers_

array([[ 0.30697647,  0.36387286,  0.08967136,  0.32764602, -0.02922655,
         0.09861813, -0.28491123, -0.03418265,  0.01645767,  0.29541324,
         0.02278577, -0.03429328,  0.15967592],
       [-0.98471673, -1.16722851, -0.2876471 , -1.05102036,  0.0937527 ,
        -0.31634647,  0.91393601,  0.10965085, -0.05279278, -0.94762429,
        -0.07309201,  0.11000573, -0.51220716]])

In [100]:
kmeans.inertia_

18203.494488062486

In [101]:
clusters = kmeans.predict(X_prep)
clusters

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [102]:
len(clusters)

1620

In [103]:
pd.Series(clusters).value_counts().sort_index()

0    1235
1     385
dtype: int64

In [104]:
data_clean['cluster'] = clusters
data_clean

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0
1,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0
2,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0
3,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0
4,0.807,0.712,1,-5.757,1,0.1000,0.5320,0.000024,0.1630,0.792,100.004,241659,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,0.657,0.591,10,-3.505,1,0.0297,0.8350,0.000000,0.2480,0.687,132.041,189347,4,0
1616,0.697,0.558,8,-6.103,1,0.0241,0.2820,0.000000,0.1000,0.831,105.966,179800,4,0
1617,0.624,0.725,2,-7.921,1,0.0315,0.0703,0.034600,0.2090,0.599,100.052,184267,4,0
1618,0.610,0.595,0,-7.027,1,0.0238,0.0654,0.000000,0.1150,0.712,104.024,177636,4,0


In [105]:
#add id back
data_clean['id']= data['id']
data_clean

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,id
0,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM
1,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM
2,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM
3,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM
4,0.807,0.712,1,-5.757,1,0.1000,0.5320,0.000024,0.1630,0.792,100.004,241659,4,0,5RAIMjdrCEjpjaR5tBATXU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,0.657,0.591,10,-3.505,1,0.0297,0.8350,0.000000,0.2480,0.687,132.041,189347,4,0,4ggabbjjuHrG06SsYnqvEs
1616,0.697,0.558,8,-6.103,1,0.0241,0.2820,0.000000,0.1000,0.831,105.966,179800,4,0,4gE9TM6eai2wzYbCga0WmD
1617,0.624,0.725,2,-7.921,1,0.0315,0.0703,0.034600,0.2090,0.599,100.052,184267,4,0,2T4FEAOUdzffXKzeyjrhPJ
1618,0.610,0.595,0,-7.027,1,0.0238,0.0654,0.000000,0.1150,0.712,104.024,177636,4,0,4g7ZMgMMZA2WPpnaeNK80l


In [106]:
#add title back
data_clean['title']= data['title']
data_clean

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,id,title
0,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año
1,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año
2,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año
3,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año
4,0.807,0.712,1,-5.757,1,0.1000,0.5320,0.000024,0.1630,0.792,100.004,241659,4,0,5RAIMjdrCEjpjaR5tBATXU,Aloha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,0.657,0.591,10,-3.505,1,0.0297,0.8350,0.000000,0.2480,0.687,132.041,189347,4,0,4ggabbjjuHrG06SsYnqvEs,Ya Me Enteré
1616,0.697,0.558,8,-6.103,1,0.0241,0.2820,0.000000,0.1000,0.831,105.966,179800,4,0,4gE9TM6eai2wzYbCga0WmD,Once Upon a Time (feat. Martina McBride)
1617,0.624,0.725,2,-7.921,1,0.0315,0.0703,0.034600,0.2090,0.599,100.052,184267,4,0,2T4FEAOUdzffXKzeyjrhPJ,True Love
1618,0.610,0.595,0,-7.027,1,0.0238,0.0654,0.000000,0.1150,0.712,104.024,177636,4,0,4g7ZMgMMZA2WPpnaeNK80l,Juntos en Tu Cama


In [107]:
#add artist back
data_clean['artist']= data['artist']
data_clean

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,id,title,artist
0,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año,Sebastian Yatra
1,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año,Sebastian Yatra
2,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año,Sebastian Yatra
3,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año,Sebastian Yatra
4,0.807,0.712,1,-5.757,1,0.1000,0.5320,0.000024,0.1630,0.792,100.004,241659,4,0,5RAIMjdrCEjpjaR5tBATXU,Aloha,Maluma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,0.657,0.591,10,-3.505,1,0.0297,0.8350,0.000000,0.2480,0.687,132.041,189347,4,0,4ggabbjjuHrG06SsYnqvEs,Ya Me Enteré,Chayín Rubio
1616,0.697,0.558,8,-6.103,1,0.0241,0.2820,0.000000,0.1000,0.831,105.966,179800,4,0,4gE9TM6eai2wzYbCga0WmD,Once Upon a Time (feat. Martina McBride),The Mavericks
1617,0.624,0.725,2,-7.921,1,0.0315,0.0703,0.034600,0.2090,0.599,100.052,184267,4,0,2T4FEAOUdzffXKzeyjrhPJ,True Love,Vince Mira
1618,0.610,0.595,0,-7.027,1,0.0238,0.0654,0.000000,0.1150,0.712,104.024,177636,4,0,4g7ZMgMMZA2WPpnaeNK80l,Juntos en Tu Cama,8 Segundos


In [108]:
data_clean['cluster'].unique()

array([0, 1], dtype=int32)

In [109]:
data_clean.to_csv('latin_clustered.csv')

3. "Predicting" which cluster id a song belongs to, would work like this: take the audio-features of the song and put it into a numpy array, let's say song_array. The audio feature order must exactly match the order that the model has learned. Call the model with KMeans.predict(song_array) to make the prediction. The outcome is only one cluster
4. The recommendation would then work like this: Take the determined cluster id from 2. and subset your spotipy dataframe to filter for this cluster with e.g. df[df["cluster_id'']==2] and get a random song back from that subset. You can take a look at pandas.DataFrame.sample

## create function which does:

- input song comes with audio features

- put into model into predict command

- get back a numpy array with 1 entriy, whihc is cluster id

- use this id to filter df by those

- do random selection of songs in cluster to recommend

### get audio features of input song

In [37]:
import getpass
client_id = getpass.getpass('client_id?')
client_secret = getpass.getpass('client_secret?')

client_id?········
client_secret?········


In [110]:
# connection to spotify API

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id ,
                                                           client_secret=client_secret))

In [128]:
# asking for input from user

track = input('Please name a song: ')
artist = input('Please name the artist of the song: ')

Please name a song: vorbeikommen
Please name the artist of the song: shelly


In [129]:
# working listcomprehension on getting the input song id

input_song_id = [track['id'] for track in sp.search(q='artist:' + artist + ' track:' + track, type='track')['tracks']['items']]
input_song_id

['0nwRKxatdg0maKm0SBZ4KG']

In [112]:
# getting audio features from track id for input song
# index 0 because we often have songs with the same title as there are several versions (remastered, with new feature, etc.)

input_song_audiofeatures = sp.audio_features(input_song_id[0])
input_song_audiofeatures

[{'danceability': 0.884,
  'energy': 0.385,
  'key': 11,
  'loudness': -14.566,
  'mode': 0,
  'speechiness': 0.354,
  'acousticness': 0.169,
  'instrumentalness': 0,
  'liveness': 0.0538,
  'valence': 0.573,
  'tempo': 82.537,
  'type': 'audio_features',
  'id': '0nwRKxatdg0maKm0SBZ4KG',
  'uri': 'spotify:track:0nwRKxatdg0maKm0SBZ4KG',
  'track_href': 'https://api.spotify.com/v1/tracks/0nwRKxatdg0maKm0SBZ4KG',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0nwRKxatdg0maKm0SBZ4KG',
  'duration_ms': 178913,
  'time_signature': 4}]

In [113]:
# transforming it into df

input_song_audiofeatures_df = pd.DataFrame(input_song_audiofeatures)
input_song_audiofeatures_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.884,0.385,11,-14.566,0,0.354,0.169,0,0.0538,0.573,82.537,audio_features,0nwRKxatdg0maKm0SBZ4KG,spotify:track:0nwRKxatdg0maKm0SBZ4KG,https://api.spotify.com/v1/tracks/0nwRKxatdg0m...,https://api.spotify.com/v1/audio-analysis/0nwR...,178913,4


In [114]:
# checking the columns

input_song_audiofeatures_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [115]:
# checking the length of the df we trained our kmeans model with

len(X_prep_df.columns)

13

In [116]:
# removing the not needed columns

input_song_audiofeatures_df_clean = input_song_audiofeatures_df.drop(['type', 'id','uri','track_href','analysis_url'], axis=1)
len(input_song_audiofeatures_df_clean.columns)

# now it has the same length as the df we trained our kmeans model with

13

In [117]:
# applying standard scaler to input song audio feature df so its the same as the df we trained model with

input_song_audiofeatures_df_prep = StandardScaler().fit_transform(input_song_audiofeatures_df_clean)

In [118]:
input_song_audiofeatures_df_prep

# its an array, thats what we need

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [119]:
# defining our column names (might be double here, could be removed, as we have it above somewhere already)

column_names = list(X_prep_df.columns)
column_names

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature']

In [120]:
# creating df from array with defined column names

input_song_audiofeatures_df_prep_df = pd.DataFrame(input_song_audiofeatures_df_prep, columns = column_names)
input_song_audiofeatures_df_prep_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### put input song into model to predict cluster

In [121]:
# put input song df into model to predict cluster

cluster_input_song = kmeans.predict(input_song_audiofeatures_df_prep_df)
len(cluster_input_song)

# len(cluster_input_song) should be 1

1

In [122]:
# putting the cluster feature back in the OG df

input_song_audiofeatures_df['cluster'] = cluster_input_song
input_song_audiofeatures_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster
0,0.884,0.385,11,-14.566,0,0.354,0.169,0,0.0538,0.573,82.537,audio_features,0nwRKxatdg0maKm0SBZ4KG,spotify:track:0nwRKxatdg0maKm0SBZ4KG,https://api.spotify.com/v1/tracks/0nwRKxatdg0m...,https://api.spotify.com/v1/audio-analysis/0nwR...,178913,4,0


### use this cluster value to filter df by those

In [123]:
# taking the cluster value from the input song

cluster_input_song = input_song_audiofeatures_df.iloc[0]['cluster']
cluster_input_song

0

In [124]:
# single out the cluster x from the songs_clustered df into new df

song_cluster_0 = data_clean[data_clean['cluster']==0]
song_cluster_0

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,id,title,artist
0,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año,Sebastian Yatra
1,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año,Sebastian Yatra
2,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año,Sebastian Yatra
3,0.715,0.622,4,-5.102,0,0.0460,0.1630,0.000000,0.0948,0.808,80.015,195053,4,0,2rCbl9naJYhaxjLsfx88uM,Pareja Del Año,Sebastian Yatra
4,0.807,0.712,1,-5.757,1,0.1000,0.5320,0.000024,0.1630,0.792,100.004,241659,4,0,5RAIMjdrCEjpjaR5tBATXU,Aloha,Maluma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,0.657,0.591,10,-3.505,1,0.0297,0.8350,0.000000,0.2480,0.687,132.041,189347,4,0,4ggabbjjuHrG06SsYnqvEs,Ya Me Enteré,Chayín Rubio
1616,0.697,0.558,8,-6.103,1,0.0241,0.2820,0.000000,0.1000,0.831,105.966,179800,4,0,4gE9TM6eai2wzYbCga0WmD,Once Upon a Time (feat. Martina McBride),The Mavericks
1617,0.624,0.725,2,-7.921,1,0.0315,0.0703,0.034600,0.2090,0.599,100.052,184267,4,0,2T4FEAOUdzffXKzeyjrhPJ,True Love,Vince Mira
1618,0.610,0.595,0,-7.027,1,0.0238,0.0654,0.000000,0.1150,0.712,104.024,177636,4,0,4g7ZMgMMZA2WPpnaeNK80l,Juntos en Tu Cama,8 Segundos


In [125]:
# creating list with all song ids from cluster 1 to make recommendation

song_cluster_0_lst = list(song_cluster_0['id'].unique())

In [130]:
# importing random to choose random track id from same cluster as input song 

import random 

recommendation_id = random.choice(song_cluster_0_lst)
recommendation_title = song_cluster_0.loc[song_cluster_0['id'] == recommendation_id , 'title'].iloc[0]
recommendation_artist = song_cluster_0.loc[song_cluster_0['id'] == recommendation_id , 'artist'].iloc[0]
print('Check this song out:', recommendation_title, 'by', recommendation_artist)

Check this song out: Trasnocho Entre Semana by Cheo
